In [1]:
import os
os.environ["http_proxy"] = "http://192.168.104.50:7890"
os.environ["https_proxy"] = "http://192.168.104.50:7890"

In [28]:
from transformers import AutoTokenizer, AutoModel
from torch import nn
import torch
import random
import linecache
import re
from d2l import torch as d2l
import pandas as pd
from tqdm.auto import tqdm
from datasets import list_datasets, load_dataset
from pprint import pprint
import h5py
import numpy as np
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# model = AutoModel.from_pretrained("bert-base-uncased")

# inputs = tokenizer("Hello world!", return_tensors="pt")
# outputs = model(**inputs)
# outputs

In [3]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
##model = AutoModel.from_pretrained('bert-base-uncased')

In [4]:
# dataset = load_dataset('sradc/chunked-wikipedia20220301en-bookcorpusopen',cache_dir='~/nvmessd/DeepLearning/datasets/',split="train")

In [5]:

# file_count=0
# text_data = []
# #save the original data
# for sample in tqdm(dataset['text']):
#     sample =re.sub(r'\s+',' ',sample.replace('\n', ' ')).strip()
#     text_data.append(sample)
#     if len(text_data) == 10000: 
#         with open(f'/home/ubutnu/nvmessd/DeepLearning/wiki_book/original/text_{file_count}.txt', 'w', encoding='utf-8') as fp: 
#             fp.write('\n'.join(text_data))
#         text_data=[]
#         file_count += 1 

In [6]:
def replace_mask(tokens,max_len):
    tokens = tokens.split(' ')
        # 为遮蔽语言模型的输入创建新的词元副本，其中输入可能包含替换的“<mask>”或随机词元
    mlm_input_tokens = [token for token in tokens]
    pred_positions_and_labels = []
    candidate_pred_positions = [i for i in range(len(tokens))]
    num_mlm_preds = max(1, round(max_len* 0.15))
    # 打乱后用于在遮蔽语言模型任务中获取15%的随机词元进行预测
    random.shuffle(candidate_pred_positions)
    for mlm_pred_position in candidate_pred_positions:
        if len(pred_positions_and_labels) >= num_mlm_preds:
            break
        masked_token = None
        # 80%的时间：将词替换为“<mask>”词元
        if random.random() < 0.8:
            masked_token = '[MASK]'
        else:
            # 10%的时间：保持词不变
           #if random.random() < 0.5:
            masked_token = tokens[mlm_pred_position]
            # 10%的时间：用随机词替换该词
            # else:
            #     masked_token = random.choice(vocab.idx_to_token)
        mlm_input_tokens[mlm_pred_position] = masked_token
        pred_positions_and_labels.append(
            (mlm_pred_position, tokens[mlm_pred_position]))
    return mlm_input_tokens, pred_positions_and_labels

In [7]:
#@save
def get_mlm_data_from_tokens(tokens,max_len):
    # candidate_pred_positions = []
    # # tokens是一个字符串列表
    # for i, token in enumerate(tokens):
    #     # 在遮蔽语言模型任务中不会预测特殊词元
    #     if token in ['<cls>', '<sep>']:
    #         continue
    #     candidate_pred_positions.append(i)
    # 遮蔽语言模型任务中预测15%的随机词元
    mlm_input_tokens, pred_positions_and_labels = replace_mask(tokens,max_len)
    pred_positions_and_labels = sorted(pred_positions_and_labels,
                                       key=lambda x: x[0])
    pred_positions = [v[0] for v in pred_positions_and_labels]
    mlm_pred_labels = [v[1] for v in pred_positions_and_labels]
    return " ".join(mlm_input_tokens), pred_positions," ".join(mlm_pred_labels)



In [36]:
class Vocab:
    def __init__(self,file):
        with open(file, 'r') as file:
            content = file.read()
        self.idx_to_token = content.split()
        self.token_to_idx = {token: idx
            for idx, token in enumerate(self.idx_to_token)}

    def __len__(self):
        return len(self.idx_to_token)
    
    def __getitem__(self,tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]
    
    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]
    
    def unk(self):  # 未知词元的索引为0
        return 100
# 使用空格分割字符串，得到单词列表
vocab=Vocab('./vocab.txt')
len(vocab)

30522

In [9]:
rand_num = [i for i in range(1, len(vocab)) if i not in (101, 102)]
# type(random.choice(rand_num))

In [10]:
def add_mask_and_label(input_ids):
    len_tokens=len(list(filter(lambda x: x != 0, input_ids)))

    num_mlm_preds = max(1, round(len_tokens* 0.15)-2)
    candidate_pred_positions=list(range(1,len_tokens-1))
    random.shuffle(candidate_pred_positions)

    pred_positions_and_labels=[]
    mlm_input_tokens=input_ids
    for mlm_pred_position in candidate_pred_positions:
        if len(pred_positions_and_labels) >= num_mlm_preds:
            break
        masked_token = None
        # 80%的时间：将词替换为“<mask>”词元
        if random.random() < 0.8:
            masked_token = 103
        else:
            # 10%的时间：保持词不变
            if random.random() < 0.5:
                masked_token = input_ids[mlm_pred_position]
            # 10%的时间：用随机词替换该词
            else:
                masked_token = random.choice(rand_num)
        pred_positions_and_labels.append(
            (mlm_pred_position,input_ids[mlm_pred_position]))
        mlm_input_tokens[mlm_pred_position] = masked_token
        #print(pred_positions_and_labels)
    return mlm_input_tokens,pred_positions_and_labels

In [18]:

# file_count=0
# text_data,pred,labels = [],[],[]
# max_len=512
# i=0
# path='/home/ubutnu/nvmessd/DeepLearning/wiki_book'
# #front = [x.replace('\n', '').split(' ') for x in paragraphs[:10]]
# # front = paragraphs[0].replace('\n', '')
# mlm_input_tokens_all,pred_positions_all,mlm_pred_labels_all,token_type_ids_all,attention_mask_all = [],[],[],[],[]
# for file_idx in tqdm(range(3353)):
#     text_path = path+'/original/text_%d.txt'%(file_idx)
#     for content_idx in range(1,10001):
#         text = linecache.getline(text_path,content_idx).strip()

#         inputs = tokenizer(text,truncation=True,padding='max_length',max_length=max_len)
#         input_ids,token_type_ids,attention_mask=inputs['input_ids'],inputs['token_type_ids'],inputs['attention_mask']

#         mlm_input_tokens, pred_positions_and_labels  = add_mask_and_label(input_ids)
        
#         pred_positions_and_labels = sorted(pred_positions_and_labels,
#                                         key=lambda x: x[0])
#         pred_positions = [v[0] for v in pred_positions_and_labels]
#         mlm_pred_labels = [v[1] for v in pred_positions_and_labels]
#         pred_positions.extend([0]*(512-len(pred_positions)))
#         mlm_pred_labels.extend([0]*(512-len(mlm_pred_labels)))

#         pred_positions_all.extend(pred_positions)
#         mlm_pred_labels_all.extend(mlm_pred_labels)

#         attention_mask_all.extend(attention_mask)
#         token_type_ids_all.extend(token_type_ids)
#         mlm_input_tokens_all.extend(mlm_input_tokens)
#     if (file_idx+1)%10==0:
#         mlm_input_path = path+'/inputm/inputm_%d.h5'%(i)
#         pred_path = path+'/pred/pred_%d.h5'%(i)
#         label_path = path+'/label/label_%d.h5'%(i)
#         attention_mask_path = path+'/attm/attm_%d.h5'%(i)
#         type_path = path+'/type/type_%d.h5'%(i)
#         with h5py.File(mlm_input_path, 'w') as h5f:
#             h5f.create_dataset('data', data=mlm_input_tokens_all) 
#         with h5py.File(type_path, 'w') as h5f:
#             h5f.create_dataset('data', data=token_type_ids_all) 
#         with h5py.File(attention_mask_path, 'w') as h5f:
#             h5f.create_dataset('data', data=attention_mask_all) 
#         with h5py.File(pred_path, 'w') as h5f:
#             h5f.create_dataset('data', data=pred_positions_all) 
#         with h5py.File(label_path, 'w') as h5f:
#             h5f.create_dataset('data', data=mlm_pred_labels_all) 
#         i+=1
#         mlm_input_tokens_all,pred_positions_all,mlm_pred_labels_all,token_type_ids_all,attention_mask_all = [],[],[],[],[]

# mlm_input_path = path+'/inputm/inputm_%d.h5'%(i)
# pred_path = path+'/pred/pred_%d.h5'%(i)
# label_path = path+'/label/label_%d.h5'%(i)
# attention_mask_path = path+'/attm/attm_%d.h5'%(i)
# type_path = path+'/type/type_%d.h5'%(i)
# with h5py.File(mlm_input_path, 'w') as h5f:
#     h5f.create_dataset('data', data=mlm_input_tokens_all) 
# with h5py.File(type_path, 'w') as h5f:
#     h5f.create_dataset('data', data=token_type_ids_all) 
# with h5py.File(attention_mask_path, 'w') as h5f:
#     h5f.create_dataset('data', data=attention_mask_all) 
# with h5py.File(pred_path, 'w') as h5f:
#     h5f.create_dataset('data', data=pred_positions_all) 
# with h5py.File(label_path, 'w') as h5f:
#     h5f.create_dataset('data', data=mlm_pred_labels_all) 
# mlm_input_tokens_all,pred_positions_all,m


# lm_pred_labels_all,token_type_ids_all,attention_mask_all = [],[],[],[],[]



  0%|          | 0/3353 [00:00<?, ?it/s]

In [2]:
import h5py
from tqdm.auto import tqdm

onepath='/home/ubutnu/nvmessd/DeepLearning/wiki_bookold'
twopath='/home/ubutnu/nvmessd/DeepLearning/wiki_book'
zippath='/home/ubutnu/nvmessd/DeepLearning/wiki_bookzip'

def one_two_zip(path,num):
    with h5py.File(onepath+path, 'r') as f:
        data=f['data'][:]
        if len(data) % num != 0:
            raise ValueError("数据长度不能被整除")
    data1=data.reshape(-1,num)
    with h5py.File(twopath+path, 'w') as new_file:
        new_file.create_dataset('data', data=data1)
    with h5py.File(zippath+path, 'w') as new_file:
        new_file.create_dataset('data', data=data1,compression='gzip',compression_opts=9)

def one_two_zipe(path,num):
    with h5py.File(onepath+path, 'r') as f:
        data=f['data'][:]
        # if len(data) % num != 0:
        #     raise ValueError("数据长度不能被整除")
    data1=data.reshape(-1,num)[:,:70]
    
    # if data1.shape[0] % 70 != 0:
    #     raise ValueError("数据长度不能被整除")
    with h5py.File(twopath+path, 'w') as new_file:
        new_file.create_dataset('data', data=data1)
    with h5py.File(zippath+path, 'w') as new_file:
        new_file.create_dataset('data', data=data1,compression='gzip',compression_opts=9)

def ctoc(path):
    with h5py.File(twopath+path, 'r') as f:
        data=f['data'][:]
    with h5py.File(zippath+path, 'w') as new_file:
        new_file.create_dataset('data', data=data,compression='gzip',compression_opts=9)

for file_idx in tqdm(range(336)):
    inputm_path = '/inputm/inputm_%d.h5'%(file_idx)
    attm_path = '/attm/attm_%d.h5'%(file_idx)
    pred_path = '/pred/pred_%d.h5'%(file_idx)
    labels_path = '/label/label_%d.h5'%(file_idx)
    weight_path = '/weight/weight_%d.h5'%(file_idx)
    ctoc(inputm_path)
    ctoc(attm_path)
    ctoc(pred_path)
    ctoc(labels_path)
    ctoc(weight_path)
    # one_two_zip(inputm_path,512)
    # one_two_zip(attm_path,512)
    # one_two_zipe(pred_path,512)
    # one_two_zipe(labels_path,512)



  0%|          | 0/336 [00:00<?, ?it/s]

In [35]:

# for file_idx in tqdm(range(336)):
#     pred_path = '/pred/pred_%d.h5'%(file_idx)
#     out = '/weight/weight_%d.h5'%(file_idx)
#     with h5py.File(twopath+pred_path, 'r') as f:
#         data=f['data'][:]
#     # non_zero_counts = np.count_nonzero(data, axis=1)
#     # weight=np.zeros_like(data)
#     # for i, count in enumerate(non_zero_counts):
#     #     weight[i, :count] = 1

#     # with h5py.File(twopath+out, 'w') as new_file:
#     #     new_file.create_dataset('data', data=weight)
#     with h5py.File(zippath+out, 'w') as new_file:
#         new_file.create_dataset('data', data=weight,compression='gzip',compression_opts=9)


  0%|          | 0/336 [00:00<?, ?it/s]

In [ ]:
# text,mask = [],[]
# for i in tqdm(range(count)):
#     with open('/home/ubutnu/nvmessd/DeepLearning/wiki_book_id/text_%d.txt'%(i), 'r', encoding='utf-8') as fp: 
#         lines = fp.read().split('\n')
#         text.append(lines)
#     with open('/home/ubutnu/nvmessd/DeepLearning/wiki_book_mask/text_%d.txt'%(i), 'r', encoding='utf-8') as fp:
#         lines = fp.read().split('\n')
#         mask.append(lines)

In [ ]:
# x=tokenizer.batch_encode_plus(text)

In [ ]:
class Wiki_book_Dataset(torch.utils.data.Dataset): 
    def __init__(self, train_path,max_len): 
        self.train_path=train_path
        self.max_len=max_len
        self.max_num_mlm_preds = round(max_len * 0.15)
        #self.rand_MASk=rand_MASk()
        # if rand_MASk is None:
        #     self.text=text
        # else:
        #     self.text=rand_MASk(text)

    
    def __len__(self): 
        # return the number of samples 
        return 33530000
 
    def __getitem__(self, i):
        file_idx,content_idx = i/10000,i%10000+1
        path_id = self.train_path+'/wiki_book_id/text_%d.txt'%(file_idx)
        path_mask = self.train_path+'/wiki_book_mask/text_%d.txt'%(file_idx)
        #path_pred = self.train_path+'/wiki_book_pred/text_pred_%d.txt'%(file_idx)

        #path_label=self.train_path+'/wiki_book_label/text_%d.txt'%(file_idx)
        id = linecache.getline(path_id,content_idx).replace('\n', '').split(' ')
        mask = linecache.getline(path_mask,content_idx).replace('\n', '').split(' ')

        all_pred_positions, all_labels= [],[]
        #pred_positions = linecache.getline(path_pred,content_idx).replace('\n', '').split(' ').remove(' ')
        # labels = linecache.getline(path_pred,content_idx).replace('\n', '').split(' ')

        id_tensor=torch.tensor(list(map(int,id)))
        mask_tensor=torch.tensor(list(map(int,mask)))
        #pred_tensor=[int(float(pred)) for pred in pred_positions]
        #labels_tensor=torch.tensor(list(map(int,labels)))
        
        # all_pred_positions.append(torch.tensor(pred_tensor + [0] * (
        #     self.max_num_mlm_preds - len(pred_positions)), dtype=torch.long))
        # all_labels.append(torch.tensor(labels_tensor + [0] * (
        #     self.max_num_mlm_preds - len(labels)), dtype=torch.long))
        #mask



        # if i <=2:
        #     print(ls)
        return id_tensor,mask_tensor#,pred_tensor#,labels_tensor
    #torch.tensor(list(map(int,text)))
    # candidate_pred_positions = []
    # # tokens是一个字符串列表
    # for i, token in enumerate(tokens):
    #     # 在遮蔽语言模型任务中不会预测特殊词元
    #     if token in ['<cls>', '<sep>']:
    #         continue
    #     candidate_pred_positions.append(i)
    # 遮蔽语言模型任务中预测15%的随机词元


In [ ]:
max_len=512
train_path='/home/ubutnu/nvmessd/DeepLearning'


In [ ]:
# train_dataset=Wiki_book_Dataset(train_path,max_len)
# train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=32,num_workers=8,shuffle=False)
# for i,(X,y,pre) in enumerate(tqdm(train_loader)):
#     #ls=X[1].split(' ')
#     #ls=list(map(int,X[1]))
#     print(X.size(),i)

    #X, y = X.to(device), y.to(device)

In [ ]:
from torch import nn

class BERTEncoder(nn.Module):
    def __init__(self, dropout=0.5):
        super(BERTEncoder, self).__init__()
        self.bert = AutoModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)
        return final_layer

In [ ]:
from torch.optim import Adam

def train(model, train_path, learning_rate, epochs,max_len):
  # 通过Dataset类获取训练和验证集
    #train, val = Dataset(train_data), Dataset(val_data)
    # DataLoader根据batch_size获取数据，训练时选择打乱样本
    train_dataset=Wiki_book_Dataset(train_path,max_len)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=8,num_workers=8,shuffle=True)
  # 判断是否使用GPU
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    # 定义损失函数和优化器
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)
    
    if use_cuda:
            model = model.cuda()
            criterion = criterion.cuda()
    # 开始进入训练循环
    for epoch_num in range(epochs):
      # 定义两个变量，用于存储训练集的准确率和损失
            total_acc_train = 0
            total_loss_train = 0
      # 进度条函数tqdm
            for input_id,mask in tqdm(train_loader):
                
                #train_label = train_label.to(device)
                mask = mask.to(device)
                input_id = input_id.to(device)
        # 通过模型得到输出
                output = model(input_id, attention_mask=mask)
                # 计算损失
        #         batch_loss = criterion(output, train_label)
        #         total_loss_train += batch_loss.item()
        #         # 计算精度
        #         acc = (output.argmax(dim=1) == train_label).sum().item()
        #         total_acc_train += acc
        # # 模型更新
        #         model.zero_grad()
        #         batch_loss.backward()
        #         optimizer.step()
            # ------ 验证模型 -----------
            # 定义两个变量，用于存储验证集的准确率和损失
            total_acc_val = 0
            total_loss_val = 0
      # 不需要计算梯度
        #     with torch.no_grad():
        #         # 循环获取数据集，并用训练好的模型进行验证
        #         for val_input, val_label in val_dataloader:
        #   # 如果有GPU，则使用GPU，接下来的操作同训练
        #             val_label = val_label.to(device)
        #             mask = val_input['attention_mask'].to(device)
        #             input_id = val_input['input_ids'].squeeze(1).to(device)
  
        #             output = model(input_id, mask)

        #             batch_loss = criterion(output, val_label)
        #             total_loss_val += batch_loss.item()
                    
        #             acc = (output.argmax(dim=1) == val_label).sum().item()
        #             total_acc_val += acc
            
            # print(
            #     f'''Epochs: {epoch_num + 1} 
            #   | Train Loss: {total_loss_train / len(train_data): .3f} 
            #   | Train Accuracy: {total_acc_train / len(train_data): .3f} 
            #   | Val Loss: {total_loss_val / len(val_data): .3f} 
            #   | Val Accuracy: {total_acc_val / len(val_data): .3f}''')  

In [ ]:
train(model,train_path,0.001,5,512)

  0%|          | 0/4191250 [00:00<?, ?it/s]